# Performing visual question answering (VQA) with ViLT

Modified from Source: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/ViLT/Inference_with_ViLT_(visual_question_answering).ipynb#scrollTo=eTEyjLn2gdlH

In this notebook, we are going to illustate visual question answering with the Vision-and-Language Transformer (ViLT). This model is very minimal: it only adds text embedding layers to an existing ViT model. It does not require any sophisticated CNN-based pipelines to feed the image to the model (unlike models like [PixelBERT](https://arxiv.org/abs/2004.00849) and [LXMERT](https://arxiv.org/abs/1908.07490)). This makes the model also much faster than previous works.

![ViLT architecture](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/vilt_architecture.jpg)

Figure taken from the original [paper](https://arxiv.org/abs/2102.03334).

HuggingFace docs: https://huggingface.co/docs/transformers/master/en/model_doc/vilt

## Set-up environment

First, we install HuggingFace Transformers.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


## Prepare image + question

Here we take our familiar cats image (of the COCO dataset) and create a corresponding question.

In [1]:
import requests
from PIL import Image 

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "How many cats are there?"
image

ImportError: dlopen(/Users/brianwu/opt/anaconda3/envs/native/lib/python3.10/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 0x0002): tried: '/Users/brianwu/opt/anaconda3/envs/native/lib/python3.10/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/brianwu/opt/anaconda3/envs/native/lib/python3.10/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so' (no such file), '/Users/brianwu/opt/anaconda3/envs/native/lib/python3.10/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64'))

In [ ]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

Let's prepare the image+text pair for the model. Here, we leverage `ViltProcessor`, which will use (behind the scenes):
* `BertTokenizerFast` to tokenize the text (and create input_ids, attention_mask, token_type_ids)
* `ViltFeatureExtractor` to resize + normalize the image (and create pixel_values and pixel_mask). 

Note that the `pixel_mask` is only relevant in case of batches, as it can be used to indicate which pixels are real/which are padding. Here we're only preparing a single example for the model, hence all values of pixel_mask will be 1.

In [ ]:
encoding = processor(image, text, return_tensors="pt")
for k,v in encoding.items():
  print(k, v.shape)

input_ids torch.Size([1, 8])
token_type_ids torch.Size([1, 8])
attention_mask torch.Size([1, 8])
pixel_values torch.Size([1, 3, 384, 512])
pixel_mask torch.Size([1, 384, 512])


## Define model

Here we load the ViLT model, fine-tuned on VQAv2, from the [hub](https://huggingface.co/dandelin/vilt-b32-finetuned-vqa).

In [ ]:
from transformers import ViltForQuestionAnswering 

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

Downloading:   0%|          | 0.00/132k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

## Forward pass

We can now forward both the `input_ids` and `pixel_values` through the model. The model outputs logits of shape (batch_size, num_labels), which in this case will be (1, 3129) - as the VQAv2 dataset has 3129 possible answers.

In [ ]:
import torch

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = torch.sigmoid(logits).argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

Predicted answer: 2


In [ ]:
import torch
import time

starting_time = time.time()
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
text = "Where is the car?"
encoding = processor(image, text, return_tensors="pt")
for k, v in encoding.items():
  print(k, v.shape)

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = torch.sigmoid(logits).argmax(-1).item()
print("Time taken:", time.time() - starting_time)
print("Predicted answer:", model.config.id2label[idx])

In [ ]:
logits.shape

torch.Size([1, 3129])

In [ ]:
logits[0,:3]

tensor([-15.9495, -18.1472, -10.3041], grad_fn=<SliceBackward0>)